In [3]:
import os
import json
import re
import pandas as pd
from openai import OpenAI
import matplotlib.pyplot as plt
import seaborn as sns

The LLM was more willing to increase the bias levels when it was stated the goal was to predict bias in order to root it out. And also potentially when asking to awnser what you would expect not what should be the case. And making sure to mention in each question what you would predict in a "action" image dataset.

gpt4 also provided more consistent outputs with this prompt than gpt-4o.

In [4]:
def CreatePrompt(action):
  action = action.replace('_', " ").replace('-', " ")
  task = "I am trying to predict what actions in a image or video dataset could have demographic biases in the samples."
  goal = "The idea is that with your help, we can know what actions to check to make sure they are not biased as predicted."
  q1 = "How much gender bias would you expect in a {} image dataset?".format(action)
  q2 = "How much age bias would you expect in a {} image dataset?".format(action)
  q3 = "How much race bias would you expect in a {} image dataset?".format(action)
  options = "Options are none, small, medium, and large. Only respond to this prompt with the chosen option for each question separated by commas."
  explain = "After providing all of the answers, in a new line explain your reasoning."
  return task + goal + q1 + q2 + q3 + options + explain

Preparation

In [5]:
key = # key here
client = OpenAI(api_key=key)
bias_data_path = # bias data path here
df = pd.read_csv(bias_data_path)
actions = df['action'].unique()

SyntaxError: invalid syntax (<ipython-input-5-e39a8c30290a>, line 1)

In [ ]:
prompts = []
for action in actions:
  prompts.append(CreatePrompt(action))

# Open the batch file for writing
with open("requests.jsonl", "w") as f:
    index = 0
    for prompt in prompts:
        request_payload = {
            "custom_id": "sample" + str(index),
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4",
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
            }
        }
        index = index + 1
        f.write(json.dumps(request_payload) + "\n")

print("✅ Batch request file 'requests.jsonl' created.")

Show Prompt

In [ ]:
prompts[345]

Test with 1 Sample

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "developer",
            "content": prompts[345]
        }
    ]
)

print(completion.choices[0].message.content)

Create Batch

In [ ]:
batch_input_file = client.files.create(
    file=open("requests.jsonl", "rb"),
    purpose="batch"
)

Show Id

In [ ]:
batch_input_file.id

Create Batch

In [ ]:
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "inference"
    }
)

Retrieve Results

Use the ID output in the create batch cell

In [ ]:
id = # batch id here
batch = client.batches.retrieve(id)
print(batch)

Get Results

Use the output_file_id printed in the retreive retrieve results cell

In [ ]:
output_file_id = # output file id here
result = client.files.content(output_file_id).content

Write To File

In [ ]:
result_file_name = "batch.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

In [ ]:
# Loading data from saved file
results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

Extract Data

In [ ]:
index = 0
final_data = []
for res in results:
    result = res['response']['body']['choices'][0]['message']['content']

    # get data
    lines = result.split('\n')
    first_line = lines[0].strip().replace('.', '')
    biases = [word.strip().lower() for word in first_line.split(',')]

    # create and add data
    sample_data = {'action': actions[index], 'gender_bias': biases[0], 'age_bias': biases[1], 'race_bias': biases[2], 'reasoning': lines[2]}
    final_data.append(sample_data)
    index = index + 1
dfb = pd.DataFrame(final_data)
dfb.to_csv('bias_prediction_data.csv', index=False)

In [ ]:
data = pd.read_csv('bias_prediction_data.csv')

Display Results

In [ ]:
# Plot the distribution of choices
dfp = data.drop(columns=['action', 'reasoning'])
sns.countplot(data=dfp.melt(value_vars=dfp.columns), x='value', hue='variable', palette='Set2')

# Titles and labels
plt.title('GPT4 Bias Prediction for actions in HAA500 dataset')
plt.xlabel('Predicted Bias')
plt.ylabel('Count')
plt.show()